#Penjelasan Singkat
Data diambil dari platform FBRef, yang mencakup statistik pemain dari lima liga top Eropa (Premier League, Bundesliga, Serie A, La Liga, dan Ligue 1) di musim 2023/2024. Data yang dikumpulkan meliputi statistik seperti Passes Attempted, Passes Completed, Pass Completion %, Progressive Passes, Tackles, Blocks, Clearances, Interceptions, Minutes Played, dan Aerial Duels Won.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

##Pengambilan data terkait statistik operan

In [2]:
url = 'https://fbref.com/en/comps/Big5/passing/players/Big-5-European-Leagues-Stats'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
tables = soup.find_all('table')

for i, table in enumerate(tables):
    print(f"Tabel {i+1}:")
    print(table.get('id'))   # Menampilkan id tabel jika ada
    print(table.get('class'))  # Menampilkan class tabel jika ada
    print('-' * 40)

Tabel 1:
stats_passing
['min_width', 'sortable', 'stats_table', 'min_width', 'shade_zero']
----------------------------------------


In [4]:
table = soup.find('table', {'id': 'stats_passing'})

In [5]:
# Ambil semua baris data dari tabel
rows = table.find_all('tr')

data_pass = []

for row in rows:
    cells = row.find_all('td')
    if len(cells) > 0:
        player_data = {
            'Player': cells[0].text.strip(),  # Tetap sebagai string
            'Position': cells[2].text.strip(),  # Tetap sebagai string
            'Age': int(cells[5].text.strip()) if cells[5].text.strip() else 0,  # Default 0 jika kosong
            'Passes Attempted': int(cells[9].text.strip().replace(',', '')) if cells[9].text.strip() else 0,  # Default 0 jika kosong
            'Passes Completed': int(cells[8].text.strip().replace(',', '')) if cells[8].text.strip() else 0,  # Default 0 jika kosong
            'Progressive Passes': int(cells[30].text.strip()) if cells[30].text.strip() else 0,
        }
        data_pass.append(player_data)

In [6]:
data_pass[0:5]

[{'Player': 'Max Aarons',
  'Position': 'DF',
  'Age': 23,
  'Passes Attempted': 581,
  'Passes Completed': 450,
  'Progressive Passes': 43},
 {'Player': 'Brenden Aaronson',
  'Position': 'MF,FW',
  'Age': 22,
  'Passes Attempted': 472,
  'Passes Completed': 365,
  'Progressive Passes': 56},
 {'Player': 'Paxten Aaronson',
  'Position': 'MF',
  'Age': 19,
  'Passes Attempted': 50,
  'Passes Completed': 41,
  'Progressive Passes': 5},
 {'Player': 'Keyliane Abdallah',
  'Position': 'FW',
  'Age': 17,
  'Passes Attempted': 1,
  'Passes Completed': 1,
  'Progressive Passes': 0},
 {'Player': 'Yunis Abdelhamid',
  'Position': 'DF',
  'Age': 35,
  'Passes Attempted': 1836,
  'Passes Completed': 1552,
  'Progressive Passes': 137}]

In [7]:
df_pass = pd.DataFrame(data_pass)

In [8]:
# Mengecek duplikat berdasarkan kolom tertentu, misalnya 'Player'
duplikat_kolom = df_pass.duplicated(subset=['Player'])

# Menampilkan baris yang duplikat berdasarkan kolom tertentu
df_duplikat_kolom = df_pass[duplikat_kolom]
df_duplikat_kolom.head()


,Player,Position,Age,Passes Attempted,Passes Completed,Progressive Passes
17,Bénie Adama Traore,"FW,MF",20,148,99,15
35,Lucien Agoume,MF,21,360,301,35
53,Sergio Akieme,DF,25,350,277,20
57,Paul Akouokou,MF,25,185,158,13
73,Mohamed Ali Cho,FW,19,268,193,25


Bisa dilihat bahwa ada kolom dengan nama yang duplikat, maka perlu dilakukan pengelompokan

In [9]:
# Menggabungkan statistik dua pemain sama yang memiliki klub berbeda di satu musim (di FBRef namanya ditulis beberapa kali, sehingga ada data duplikat)
# Mengatasinya dengan mengelompokkan berdasarkan kolom 'Player'
df_pass = df_pass.groupby('Player').agg({
    'Position': 'first',
    'Age': 'mean',  # Rata-rata umur jika berbeda
    'Passes Attempted': 'sum',
    'Passes Completed': 'sum',
    'Progressive Passes': 'sum'
}).reset_index()

In [10]:
# Mengecek duplikat berdasarkan kolom tertentu, misalnya 'Player'
duplikat_kolom = df_pass.duplicated(subset=['Player'])

# Menampilkan baris yang duplikat berdasarkan kolom tertentu
df_duplikat_kolom = df_pass[duplikat_kolom]
df_duplikat_kolom.head()


,Player,Position,Age,Passes Attempted,Passes Completed,Progressive Passes


Bisa dilihat sudah tidak terdapat pemain yang duplikat

##Pengambilan data terkait statistik bertahan

In [11]:
url = 'https://fbref.com/en/comps/Big5/defense/players/Big-5-European-Leagues-Stats'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [12]:
tables = soup.find_all('table')

for i, table in enumerate(tables):
    print(f"Tabel {i+1}:")
    print(table.get('id'))   # Menampilkan id tabel jika ada
    print(table.get('class'))  # Menampilkan class tabel jika ada
    print('-' * 40)

Tabel 1:
stats_defense
['min_width', 'sortable', 'stats_table', 'min_width', 'shade_zero']
----------------------------------------


In [13]:
table = soup.find('table', {'id': 'stats_defense'})

In [14]:
# Ambil semua baris data dari tabel
rows = table.find_all('tr')

data_def = []

for row in rows:
    cells = row.find_all('td')
    if len(cells) > 0:
        player_data = {
            'Player': cells[0].text.strip(),
            'Tackles': int(cells[8].text.strip()) if cells[8].text.strip() else 0,
            'Blocks': int(cells[17].text.strip()) if cells[17].text.strip() else 0,
            'Clearances': int(cells[22].text.strip()) if cells[22].text.strip() else 0,
            'Interceptions': int(cells[20].text.strip()) if cells[20].text.strip() else 0,
        }
        data_def.append(player_data)

In [15]:
data_def[0:5]

[{'Player': 'Max Aarons',
  'Tackles': 29,
  'Blocks': 9,
  'Clearances': 27,
  'Interceptions': 8},
 {'Player': 'Brenden Aaronson',
  'Tackles': 32,
  'Blocks': 26,
  'Clearances': 4,
  'Interceptions': 2},
 {'Player': 'Paxten Aaronson',
  'Tackles': 2,
  'Blocks': 2,
  'Clearances': 0,
  'Interceptions': 0},
 {'Player': 'Keyliane Abdallah',
  'Tackles': 0,
  'Blocks': 0,
  'Clearances': 0,
  'Interceptions': 0},
 {'Player': 'Yunis Abdelhamid',
  'Tackles': 64,
  'Blocks': 51,
  'Clearances': 109,
  'Interceptions': 39}]

In [16]:
df_def = pd.DataFrame(data_def)

In [17]:
# Menggabungkan statistik dua pemain sama yang memiliki klub berbeda di satu musim (di FBRef namanya ditulis beberapa kali, sehingga ada data duplikat)
# Mengatasinya dengan mengelompokkan berdasarkan kolom 'Player'
df_def = df_def.groupby('Player').agg({
    'Tackles': 'sum',
    'Blocks': 'sum',
    'Clearances': 'sum',
    'Interceptions': 'sum'
}).reset_index()

##Pengambilan data terkait statistik menit bermain

In [18]:
url = 'https://fbref.com/en/comps/Big5/playingtime/players/Big-5-European-Leagues-Stats'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [19]:
tables = soup.find_all('table')

for i, table in enumerate(tables):
    print(f"Tabel {i+1}:")
    print(table.get('id'))   # Menampilkan id tabel jika ada
    print(table.get('class'))  # Menampilkan class tabel jika ada
    print('-' * 40)

Tabel 1:
stats_playing_time
['min_width', 'sortable', 'stats_table', 'min_width', 'shade_zero']
----------------------------------------


In [20]:
table = soup.find('table', {'id': 'stats_playing_time'})

In [21]:
# Ambil semua baris data dari tabel
rows = table.find_all('tr')

data_playtime = []

for row in rows:
    cells = row.find_all('td')
    if len(cells) > 0:
        player_data = {
            'Player': cells[0].text.strip(),
            'Minutes Played': int(cells[8].text.strip().replace(',', '')) if cells[8].text.strip() else 0,
        }
        data_playtime.append(player_data)

In [22]:
data_playtime[0:5]

[{'Player': 'Max Aarons', 'Minutes Played': 1237},
 {'Player': 'Brenden Aaronson', 'Minutes Played': 1267},
 {'Player': 'Paxten Aaronson', 'Minutes Played': 101},
 {'Player': 'James Abankwah', 'Minutes Played': 0},
 {'Player': 'Keyliane Abdallah', 'Minutes Played': 4}]

In [23]:
df_playtime = pd.DataFrame(data_playtime)

In [24]:
# Menggabungkan statistik dua pemain sama yang memiliki klub berbeda di satu musim (di FBRef namanya ditulis beberapa kali, sehingga ada data duplikat)
# Mengatasinya dengan mengelompokkan berdasarkan kolom 'Player'
df_playtime = df_playtime.groupby('Player').agg({
    'Minutes Played': 'sum'
}).reset_index()

##Pengambilan data terkait statistik duel udara

In [25]:
url = 'https://fbref.com/en/comps/Big5/misc/players/Big-5-European-Leagues-Stats'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [26]:
tables = soup.find_all('table')

for i, table in enumerate(tables):
    print(f"Tabel {i+1}:")
    print(table.get('id'))   # Menampilkan id tabel jika ada
    print(table.get('class'))  # Menampilkan class tabel jika ada
    print('-' * 40)

Tabel 1:
stats_misc
['min_width', 'sortable', 'stats_table', 'min_width', 'shade_zero']
----------------------------------------


In [27]:
table = soup.find('table', {'id': 'stats_misc'})

In [28]:
# Ambil semua baris data dari tabel
rows = table.find_all('tr')

data_duel = []

for row in rows:
    cells = row.find_all('td')
    if len(cells) > 0:
        player_data = {
            'Player': cells[0].text.strip(),
            'Aerial Duels Won': int(cells[21].text.strip()) if cells[21].text.strip() else 0,
        }
        data_duel.append(player_data)

In [29]:
data_duel[0:5]

[{'Player': 'Max Aarons', 'Aerial Duels Won': 5},
 {'Player': 'Brenden Aaronson', 'Aerial Duels Won': 13},
 {'Player': 'Paxten Aaronson', 'Aerial Duels Won': 3},
 {'Player': 'Keyliane Abdallah', 'Aerial Duels Won': 0},
 {'Player': 'Yunis Abdelhamid', 'Aerial Duels Won': 61}]

In [30]:
df_duel = pd.DataFrame(data_duel)

In [31]:
# Menggabungkan statistik dua pemain sama yang memiliki klub berbeda di satu musim (di FBRef namanya ditulis beberapa kali, sehingga ada data duplikat)
# Mengatasinya dengan mengelompokkan berdasarkan kolom 'Player'
df_duel = df_duel.groupby('Player').agg({
    'Aerial Duels Won': 'sum'
}).reset_index()

## Menggabungkan keempat dataframe berdasarkan nama pemain

In [32]:
# Gabungkan df_pass dengan df_def
df_5leagues = pd.merge(df_pass, df_def, on='Player', how='inner')

# Gabungkan hasil dengan df_playtime
df_5leagues = pd.merge(df_5leagues, df_playtime, on='Player', how='inner')

# Gabungkan hasil dengan df_duel
df_5leagues = pd.merge(df_5leagues, df_duel, on='Player', how='inner')

In [33]:
df_5leagues.head()

,Player,Position,Age,Passes Attempted,Passes Completed,Progressive Passes,Tackles,Blocks,Clearances,Interceptions,Minutes Played,Aerial Duels Won
0,Aaron Cresswell,"DF,FW",33.0,308,254,26,3,5,7,3,436,6
1,Aaron Hickey,DF,21.0,277,243,21,16,7,10,3,713,1
2,Aaron Malouda,FW,17.0,0,0,0,0,0,0,0,1,0
3,Aaron Ramsdale,GK,25.0,188,131,2,0,0,3,0,540,0
4,Aaron Ramsey,"MF,FW",20.0,151,118,8,17,11,10,1,527,4


## Filtering data
Data difilter untuk hanya menyertakan pemain berposisi gelandang yang bermain reguler (bermain lebih dari 1000 menit) dan berusia tidak lebih dari 25 tahun untuk fokus pada potensi jangka panjang.

In [34]:
# Filter berdasarkan posisi MF
df_mf = df_5leagues[df_5leagues['Position'].str.contains('MF', na=False)]

# Filter berdasarkan umur
df_mf = df_mf[df_mf['Age'] <= 25]

# Filter berdasarkan menit bermain
df_mf = df_mf[df_mf['Minutes Played'] >= 1000]

In [35]:
df_mf.head()

,Player,Position,Age,Passes Attempted,Passes Completed,Progressive Passes,Tackles,Blocks,Clearances,Interceptions,Minutes Played,Aerial Duels Won
15,Abdou Harroui,"MF,FW",25.0,383,303,38,23,19,8,5,1019,12
37,Adam Wharton,MF,19.0,596,464,79,48,17,24,20,1297,7
41,Ademola Lookman,"FW,MF",25.0,838,635,97,20,19,0,9,1894,18
61,Aimar Oroz,MF,21.0,1035,839,103,43,27,11,16,2339,19
63,Ainsley Maitland-Niles,"DF,MF",25.0,796,663,76,34,16,29,17,1432,18


In [36]:
# Menambahkan kolom Pass Completion %
df_mf['Pass Completion %'] = (df_mf['Passes Completed'] / df_mf['Passes Attempted'] * 100).map('{:.1f}'.format)

In [37]:
df_mf.head()

,Player,Position,Age,Passes Attempted,Passes Completed,Progressive Passes,Tackles,Blocks,Clearances,Interceptions,Minutes Played,Aerial Duels Won,Pass Completion %
15,Abdou Harroui,"MF,FW",25.0,383,303,38,23,19,8,5,1019,12,79.1
37,Adam Wharton,MF,19.0,596,464,79,48,17,24,20,1297,7,77.9
41,Ademola Lookman,"FW,MF",25.0,838,635,97,20,19,0,9,1894,18,75.8
61,Aimar Oroz,MF,21.0,1035,839,103,43,27,11,16,2339,19,81.1
63,Ainsley Maitland-Niles,"DF,MF",25.0,796,663,76,34,16,29,17,1432,18,83.3


In [38]:
df_mf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 343 entries, 15 to 2693
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Player              343 non-null    object 
 1   Position            343 non-null    object 
 2   Age                 343 non-null    float64
 3   Passes Attempted    343 non-null    int64  
 4   Passes Completed    343 non-null    int64  
 5   Progressive Passes  343 non-null    int64  
 6   Tackles             343 non-null    int64  
 7   Blocks              343 non-null    int64  
 8   Clearances          343 non-null    int64  
 9   Interceptions       343 non-null    int64  
 10  Minutes Played      343 non-null    int64  
 11  Aerial Duels Won    343 non-null    int64  
 12  Pass Completion %   343 non-null    object 
dtypes: float64(1), int64(9), object(3)
memory usage: 37.5+ KB


In [39]:
# Mengecek duplikat berdasarkan kolom tertentu, misalnya 'Player'
duplikat_kolom = df_mf.duplicated(subset=['Player'])

# Menampilkan baris yang duplikat berdasarkan kolom tertentu
df_duplikat_kolom = df_mf[duplikat_kolom]

print(df_duplikat_kolom)

Empty DataFrame
Columns: [Player, Position, Age, Passes Attempted, Passes Completed, Progressive Passes, Tackles, Blocks, Clearances, Interceptions, Minutes Played, Aerial Duels Won, Pass Completion %]
Index: []


Bisa dilihat bahwa dataframe nya tidak mengandung nama pemain yang duplikat. Langkah berikutnya adalah penghapusan beberapa kolom yang tidak akan digunakan pada proses selanjutnya

## Penghapusan kolom yang tidak akan digunakan pada proses clustering
Kolom-kolom yang dihapus adalah kolom-kolom yang digunakan pada proses filtering dan kolom "Passes Completed" yang sudah tidak perlu digunakan karena sudah ada kolom "Pass Completion %"

In [40]:
kolom_yang_dihapus = ['Position', 'Age', 'Minutes Played', 'Passes Completed']

# Menghapus kolom-kolom tersebut
df_mf = df_mf.drop(columns=kolom_yang_dihapus)

In [41]:
df_mf.head()

,Player,Passes Attempted,Progressive Passes,Tackles,Blocks,Clearances,Interceptions,Aerial Duels Won,Pass Completion %
15,Abdou Harroui,383,38,23,19,8,5,12,79.1
37,Adam Wharton,596,79,48,17,24,20,7,77.9
41,Ademola Lookman,838,97,20,19,0,9,18,75.8
61,Aimar Oroz,1035,103,43,27,11,16,19,81.1
63,Ainsley Maitland-Niles,796,76,34,16,29,17,18,83.3


## Menyimpan dataframe ke dalam bentuk file excel

In [42]:
df_mf.to_excel('midfielderstop5leagues.xlsx', index=False)